# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.916284 
 0.586171 
 0.312024 
 0.553864 
 0.514377 
 0.513453 
 0.508622 
 0.613465 
 0.404851 
 0.743358 
 0.198943 
 0.670542 
 0.0108015
 ⋮        
 0.959275 
 0.732448 
 0.67355  
 0.850121 
 0.750821 
 0.632139 
 0.469158 
 0.944186 
 0.897495 
 0.606113 
 0.0696236
 0.275289 

In [2]:
sum(a)   

5.00147136650858e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [3]:
using Compat
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [4]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [5]:
c_sum(a)

5.001471366508968e6

In [6]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbol

true

In [7]:
≈  # alias for the `isapprox` function

isapprox (generic function with 3 methods)

In [8]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true
```


We can now benchmark the C code directly from Julia:

In [9]:
c_bench = @benchmark c_sum($a) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.526 ms (0.00% GC)
  median time:      7.534 ms (0.00% GC)
  mean time:        7.573 ms (0.00% GC)
  maximum time:     8.084 ms (0.00% GC)
  --------------
  samples:          660
  evals/sample:     1

In [10]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 7.526249 msec


In [11]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "C" => 7.52625

In [12]:
using Plots
gr()

Plots.GRBackend()

In [13]:
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 7.52 
 
 
 7.54 
 
 
 7.56 
 
 
 7.58 
 
 
 7.60 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 milliseconds 
 
 
 count

# 2. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [15]:
using PyCall

In [16]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [17]:
pysum(a)

5.001471366508968e6

In [18]:
pysum(a) ≈ sum(a)

true

In [19]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     101.003 ms (0.00% GC)
  median time:      101.089 ms (0.00% GC)
  mean time:        101.181 ms (0.00% GC)
  maximum time:     103.839 ms (0.00% GC)
  --------------
  samples:          50
  evals/sample:     1

In [20]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "C"               => 7.52625
  "Python built-in" => 101.003

# 3. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [21]:
using Conda 
#Conda.add("numpy")

In [22]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  22
  --------------
  minimum time:     3.763 ms (0.00% GC)
  median time:      3.786 ms (0.00% GC)
  mean time:        3.792 ms (0.00% GC)
  maximum time:     3.994 ms (0.00% GC)
  --------------
  samples:          1317
  evals/sample:     1

In [23]:
numpy_sum(apy_numpy) # python thing

5.001471366508571e6

In [24]:
numpy_sum(apy_numpy) ≈ sum(a)

true

In [25]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "C"               => 7.52625
  "Python numpy"    => 3.76293
  "Python built-in" => 101.003

# 4. Python, hand-written 

In [26]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x7f8d68008578>

In [27]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     310.804 ms (0.00% GC)
  median time:      311.386 ms (0.00% GC)
  mean time:        313.260 ms (0.00% GC)
  maximum time:     321.447 ms (0.00% GC)
  --------------
  samples:          16
  evals/sample:     1

In [28]:
sum_py(apy_list)

5.001471366508968e6

In [29]:
sum_py(apy_list) ≈ sum(a)

true

In [30]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                   => 7.52625
  "Python numpy"        => 3.76293
  "Python hand-written" => 310.804
  "Python built-in"     => 101.003

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [31]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [32]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.715 ms (0.00% GC)
  median time:      3.726 ms (0.00% GC)
  mean time:        3.731 ms (0.00% GC)
  maximum time:     3.921 ms (0.00% GC)
  --------------
  samples:          1339
  evals/sample:     1

In [33]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                   => 7.52625
  "Python numpy"        => 3.76293
  "Python hand-written" => 310.804
  "Python built-in"     => 101.003
  "Julia built-in"      => 3.7149

# 6. Julia (hand-written) 

In [34]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [35]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.529 ms (0.00% GC)
  median time:      7.542 ms (0.00% GC)
  mean time:        7.662 ms (0.00% GC)
  maximum time:     9.131 ms (0.00% GC)
  --------------
  samples:          653
  evals/sample:     1

In [36]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 7.52625
  "Python numpy"        => 3.76293
  "Julia hand-written"  => 7.52924
  "Python hand-written" => 310.804
  "Python built-in"     => 101.003
  "Julia built-in"      => 3.7149

# Summary

In [37]:
for (key, value) in sort(collect(d))
    println(rpad(key, 20, "."), lpad(round(value, 1), 8, "."))
end

C........................7.5
Julia built-in...........3.7
Julia hand-written.......7.5
Python built-in........101.0
Python hand-written....310.8
Python numpy.............3.8


In [38]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

Julia built-in............3.71
Python numpy..............3.76
C.........................7.53
Julia hand-written........7.53
Python built-in..........101.0
Python hand-written......310.8
